In [ ]:
import requests
import re
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np



In [ ]:

### This part is only for yahoo finance
url=("https://finance.yahoo.com/rss/industry?")
r1=requests.get(url)
coverpage = str(r1.content)


yahoolinks=re.findall("<link>(.*?)</link>", coverpage)

yahootext = re.findall("&gt(.*?)&lt", coverpage)
yahootext=yahootext[15:]
for i in yahootext:
    if i.startswith(";<"):
        yahootext.remove(i)
        
for i in yahootext:
    if i==";":
        yahootext.remove(i)
yahootext=list(set(yahootext))


### This part is for nytimes

links=["https://rss.nytimes.com/services/xml/rss/nyt/Business.xml", "https://rss.nytimes.com/services/xml/rss/nyt/Economy.xml"]
ny=""
for url in links:
    r2 = requests.get(url)
    ny+=r2.text


nylinks=re.findall("<link>(.*?)</link>", ny)
nylinks=nylinks[2:]
nytext=[]

nylinks=list(set(nylinks))


headers={"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362"}

for url in nylinks:
    
    session = requests.Session()
    session.auth = ("XXXXX", "XXXXX")

    auth = session.post("https://www.nytimes.com/")
    r3=session.get(url,headers=headers)
    temp=re.findall("<p class=\"css-158dogj evys1bk0\">(.*?)</p>", r3.text)  
    temp=temp[1:]
    string=""
    for i in temp:
        string+=i
    string.join("/n")
    nytext.append(string)

nytext=list(set(nytext))

### This section is for forbes

source = urlopen('https://www.forbes.com/markets')
soup = BeautifulSoup(source,'lxml')

lstforbesmarket = []
for i in soup.findAll('a', {'class': "section-pick__title" }):
    title = i.text
    link = i['href']
    lstforbesmarket.append(link)

forbestext=[]

for url in lstforbesmarket:

    
    r4=requests.get(url,headers=headers)

    temp=re.findall("<p>(.*?)</p>", r4.text)
    temp=temp[2:]

    
    string=""
    for i in temp:
        string+=i
    string.join("/n")
    forbestext.append(string)

In [ ]:
import torch
import torchvision
from transformers import pipeline, AutoModelForTokenClassification, AutoTokenizer
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA


summarizer = pipeline("summarization", framework="pt", device=0)


### helper function for dividing the text to smaller size so that it could be fed into the model
def chunkstring(string, length):
    return (string[0+i:length+i] for i in range(0, len(string), length))

yahoo=""
ny=""
forbes=""

for i in yahootext:
    yahoo+=(" "+i)
for i in nytext:
    ny+=(" "+i)
for i in forbestext:
    forbes+=(" "+i)
### This is all the text that needs to be summarized
text=yahoo+" "+ny+" "+forbes

### We chunk the text into smaller parts
chunkedstring=list(chunkstring(text, 1024))





In [ ]:
from flask import Flask, request, render_template



url = 'http://localhost:3000/predict'

app = Flask(__name__, template_folder="templates")

# Load the model

@app.route('/', methods=['GET','POST'])


def home():
    
    global sentimentscore
    global answer
    global counter
    ### With each click we take batches of size 20 from the chunkedstring and pass it through the summarization pipeline
    ### and give it a sentiment score. Vader sentiment score is between -1 and 1 so we scale it to be between 0 and 100. 
    ### Witch each click the sentiment score is averaged over all the previous clicks.
    if request.method == 'POST':
        batch=20
        
        for i in range(0,batch):
            temp=summarizer(chunkedstring[counter+i], min_length=0, max_length=20)[0]['summary_text']
            score=SIA().polarity_scores(chunkedstring[counter+i])["compound"]
            sentimentscore.append(score)
            answer.append(temp)
        counter+=20
        return render_template('demo.html', sentiment= answer[counter-20]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-19])))),
                                  sentiment2= answer[counter-19]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-18])))),
                              sentiment3= answer[counter-18]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-17])))),
                              sentiment4= answer[counter-17]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-16])))),
                              sentiment5= answer[counter-16]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-15])))),
                              sentiment6= answer[counter-15]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-14])))),
                              sentiment7= answer[counter-14]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-13])))),
                              sentiment8= answer[counter-13]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-12])))),
                              sentiment9= answer[counter-12]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-11])))),
                              sentiment10= answer[counter-11]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-10])))),
                              sentiment11= answer[counter-10]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-9])))),
                              sentiment12= answer[counter-9]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-8])))),
                              sentiment13= answer[counter-8]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-7])))),
                              sentiment14= answer[counter-7]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-6])))),
                              sentiment15= answer[counter-6]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-5])))),
                              sentiment16= answer[counter-5]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-4])))),
                              sentiment17= answer[counter-4]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-3])))),
                              sentiment18= answer[counter-3]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-2])))),
                              sentiment19= answer[counter-2]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter-1])))),
                              sentiment20= answer[counter-1]+". " +"The sentiment score so far is "+ str(int(50*(1+np.mean(sentimentscore[:counter])))))
        

        
        
    return render_template('demo.html', sentiment='', sentiment2='', sentiment3='', sentiment4='', sentiment5='', sentiment6='', sentiment7='',
                           sentiment8='', sentiment9='', sentiment10='', sentiment11='', sentiment12='', sentiment13='', sentiment14='', 
                           sentiment15='', sentiment16='', sentiment17='', sentiment18='', sentiment19='', sentiment20='')
        
        
    
if __name__ == '__main__':
    app.run(port=3000, debug=False)

